In [1]:
import pickle as p
import os

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# from tensorflow.examples.tutorials.mnist import input_data
# mnist = input_data.read_data_sets("MNIST_data/",one_hot=False)

def strip_first_col(fname, delimiter=None):
    with open(fname, 'rb') as fin:
        for line in fin:
            try:
               yield line.split(delimiter, 1)[1]
            except IndexError:
               continue


In [2]:
train_data = np.load("./train_data.npy")
train_label = np.load("./train_label.npy")
test_data = np.load("./test_data.npy")
test_label = np.load("./test_label.npy")

## autoencoder pretrain

In [2]:
CAPACITY = 30000
training_epochs = 10
learning_rate = 0.01
display_step = 1
examples_to_show = 10
n_input = 22283 #Need to change to 22283

# tf Graph input (only pictures)
X = tf.placeholder("float",[None,n_input])
# X = get_batch(data,BATCH_SIZE,CAPACITY)

#need to change to 4 layers 
n_hidden_1 = 1
# n_hidden_2 = 22283
# n_hidden_3 = 22283
# n_hidden_4 = 22283

weights= {
    'encoder_h1': tf.get_variable('eh1', [n_input,n_hidden_1],tf.float32, initializer=tf.contrib.layers.xavier_initializer()),
#     'encoder_h2': tf.get_variable('eh2', [n_hidden_1,n_hidden_2],tf.float32, initializer=tf.contrib.layers.xavier_initializer()),
#     'encoder_h3': tf.get_variable('eh3', [n_hidden_2,n_hidden_3],tf.float32, initializer=tf.contrib.layers.xavier_initializer()),
#     'encoder_h4': tf.get_variable('eh4', [n_hidden_3,n_hidden_4],tf.float32, initializer=tf.contrib.layers.xavier_initializer()),
#     'decoder_h1': tf.get_variable('dh1', [n_hidden_4,n_hidden_3],tf.float32, initializer=tf.contrib.layers.xavier_initializer()),
#     'decoder_h2': tf.get_variable('dh2', [n_hidden_3,n_hidden_2],tf.float32, initializer=tf.contrib.layers.xavier_initializer()),
#     'decoder_h3': tf.get_variable('dh3', [n_hidden_2,n_hidden_1],tf.float32, initializer=tf.contrib.layers.xavier_initializer()),
    'decoder_h4': tf.get_variable('dh4', [n_hidden_1,n_input],tf.float32, initializer=tf.contrib.layers.xavier_initializer()),
#     'encoder_h2': tf.Variable(tf.truncated_normal([n_hidden_1,n_hidden_2], stddev=0.1)),
#     'encoder_h3': tf.Variable(tf.truncated_normal([n_hidden_2,n_hidden_3], stddev=0.1)),
#     'encoder_h4': tf.Variable(tf.truncated_normal([n_hidden_3,n_hidden_4], stddev=0.1)),
#     'decoder_h1': tf.Variable(tf.truncated_normal([n_hidden_4,n_hidden_3], stddev=0.1)),
#     'decoder_h2': tf.Variable(tf.truncated_normal([n_hidden_3,n_hidden_2], stddev=0.1)),
#     'decoder_h3': tf.Variable(tf.truncated_normal([n_hidden_2,n_hidden_1], stddev=0.1)),
#     'decoder_h4': tf.Variable(tf.truncated_normal([n_hidden_1,n_input], stddev=0.1)),
}

biases = {
    'encoder_b1': tf.Variable(tf.random_normal([n_hidden_1])),
#     'encoder_b2': tf.Variable(tf.random_normal([n_hidden_2])),
#     'encoder_b3': tf.Variable(tf.random_normal([n_hidden_3])),
#     'encoder_b4': tf.Variable(tf.random_normal([n_hidden_4])),
#     'decoder_b1': tf.Variable(tf.random_normal([n_hidden_3])),
#     'decoder_b2': tf.Variable(tf.random_normal([n_hidden_2])),
#     'decoder_b3': tf.Variable(tf.random_normal([n_hidden_1])),
    'decoder_b4': tf.Variable(tf.random_normal([n_input])),
}
# weightse1_summary = tf.summary.histogram("weights_e1", weights['encoder_h1'])
# weightse2_summary = tf.summary.histogram("weights_e2", weights['encoder_h2'])
# weightse3_summary = tf.summary.histogram("weights_e3", weights['encoder_h3'])
# weightsd1_summary = tf.summary.histogram("weights_d1", weights['decoder_h1'])
# weightsd2_summary = tf.summary.histogram("weights_d2", weights['decoder_h1'])
# weightsd3_summary = tf.summary.histogram("weights_d3", weights['decoder_h1'])

def encoder(x):  
    layer_1 = tf.nn.tanh(tf.add(tf.matmul(x, weights['encoder_h1']),  
                                   biases['encoder_b1']))  
#     layer_2 = tf.nn.tanh(tf.add(tf.matmul(layer_1, weights['encoder_h2']),  
#                                    biases['encoder_b2']))  
#     layer_3 = tf.nn.tanh(tf.add(tf.matmul(layer_2, weights['encoder_h3']),
#                                    biases['encoder_b3']))
#     layer_4 = tf.nn.tanh(tf.add(tf.matmul(layer_3, weights['encoder_h4']),
#                                    biases['encoder_b4']))
    return layer_1

def decoder(x):  
    layer_1 = tf.nn.tanh(tf.add(tf.matmul(x, weights['decoder_h4']),  
                                   biases['decoder_b4']))  
#     layer_2 = tf.nn.tanh(tf.add(tf.matmul(layer_1, weights['decoder_h2']),  
#                                    biases['decoder_b2']))  
#     layer_3 = tf.nn.tanh(tf.add(tf.matmul(layer_2, weights['decoder_h3']),
#                                    biases['decoder_b3']))
#     layer_4 = tf.nn.tanh(tf.add(tf.matmul(layer_3, weights['decoder_h4']),
#                                    biases['decoder_b4']))
    return layer_1

encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

y_pred = decoder_op
y_true = X

In [3]:
cost = tf.reduce_mean(tf.pow(y_true-y_pred,2))
# cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_pred, labels=y_true))
cost_summary = tf.summary.scalar('Cost', cost)  
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

sess = tf.Session()
if int((tf.__version__).split('.')[1]) < 12 and int((tf.__version__).split('.')[0]) < 1:  
    init = tf.initialize_all_variables()  
else:  
    init = tf.global_variables_initializer()  
sess.run(init)  

In [4]:
index = np.random.permutation(raw_data.shape[0])

In [5]:
def get_batch_np(itert, batchsize):
    batch_index = index[itert*batchsize:(itert+1)*batchsize]
    batch_data = raw_data[batch_index]
    return batch_data

In [12]:
BATCH_SIZE = 64
learning_rate = 2
training_epochs = 200
# 首先计算总批数，保证每次循环训练集中的每个样本都参与训练，不同于批量训练  
total_batch = int(raw_data.shape[0]/BATCH_SIZE) #总批数  

merged_summary_op = tf.summary.merge_all()
summary_writer = tf.summary.FileWriter('./tmp/logs', sess.graph)
total_step = 0

for epoch in range(training_epochs):  
    index = np.random.permutation(raw_data.shape[0])
    for i in range(total_batch):  
        total_step += 1
        batch_xs= get_batch_np(i, BATCH_SIZE)  # max(x) = 1, min(x) = 0  
        
        # Run optimization op (backprop) and cost op (to get loss value)  
        _, c = sess.run([optimizer, cost], feed_dict={X: batch_xs})
        
        
#         summary_str = sess.run(merged_summary_op, feed_dict={X: batch_xs})
#         summary_writer.add_summary(summary_str, total_step)
#         summary_writer.add_graph(sess.graph)
        
        if i % 10 == 0 or i<10:
            print "Iter: %d" % i, "cost =", "{:.9f}".format(c) 
    if epoch % display_step == 0:  
        record = open("train_loss.txt", "a+")
        record.write("epoch:"+str(epoch)+" loss:"+"{:.9f}\n".format(c))
        record.close()
        print "Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(c)  
        init_op = tf.global_variables_initializer()
        saver = tf.train.Saver()
        with tf.Session() as sesstemp:
            sesstemp.run(init_op)
            saver_path = saver.save(sesstemp, "model.ckpt")
            print "Model saved in file: ", saver_path
print("Optimization Finished!")  

Iter: 0 cost = 1132690.875000000
Iter: 1 cost = 1114782.625000000
Iter: 2 cost = 1127686.625000000
Iter: 3 cost = 1148600.500000000
Iter: 4 cost = 1138921.375000000
Iter: 5 cost = 1130688.500000000
Iter: 6 cost = 1113523.750000000
Iter: 7 cost = 1128154.625000000
Iter: 8 cost = 1086702.375000000
Iter: 9 cost = 1127517.375000000
Iter: 10 cost = 1128266.500000000
Iter: 20 cost = 1131665.625000000
Iter: 30 cost = 1117142.125000000
Iter: 40 cost = 1119001.500000000
Iter: 50 cost = 1129301.250000000
Iter: 60 cost = 1134886.125000000
Iter: 70 cost = 1128734.875000000
Iter: 80 cost = 1129180.875000000
Iter: 90 cost = 1092207.375000000
Iter: 100 cost = 1138906.250000000
Iter: 110 cost = 1118319.875000000
Iter: 120 cost = 1135590.625000000
Iter: 130 cost = 1112696.375000000
Iter: 140 cost = 1127809.500000000
Iter: 150 cost = 1125737.500000000
Iter: 160 cost = 1130493.375000000
Iter: 170 cost = 1121293.000000000
Iter: 180 cost = 1126222.000000000
Iter: 190 cost = 1100796.375000000
Iter: 200 cost

KeyboardInterrupt: 

## classify 

In [3]:
CAPACITY = 30000
training_epochs = 10
learning_rate = tf.placeholder(tf.float32, shape=[])
display_step = 1
examples_to_show = 10
n_input = 22283 #Need to change to 22283

# tf Graph input (only pictures)

X = tf.placeholder("float",[None,n_input])
y = tf.placeholder("float", shape=[None, 2])  
# X = get_batch(data,BATCH_SIZE,CAPACITY)

#need to change to 4 layers 
n_hidden_1 = 200
n_hidden_2 = 150
n_hidden_3 = 100
n_hidden_4 = 42
n_class = 2

weights= {
    'encoder_h1': tf.Variable(tf.truncated_normal([n_input,n_hidden_1], stddev=0.1), name='encoder_h1'),
    'encoder_h2': tf.Variable(tf.truncated_normal([n_hidden_1,n_hidden_2], stddev=0.1), name='encoder_h2'),
    'encoder_h3': tf.Variable(tf.truncated_normal([n_hidden_2,n_hidden_3], stddev=0.1), name='encoder_h3'),
    'encoder_h4': tf.Variable(tf.truncated_normal([n_hidden_3,n_hidden_4], stddev=0.1), name='encoder_h4'),
    'decoder_h1': tf.Variable(tf.truncated_normal([n_hidden_4,n_hidden_3], stddev=0.1), name='decoder_h1'),
    'decoder_h2': tf.Variable(tf.truncated_normal([n_hidden_3,n_hidden_2], stddev=0.1), name='decoder_h2'),
    'decoder_h3': tf.Variable(tf.truncated_normal([n_hidden_2,n_hidden_1], stddev=0.1), name='decoder_h3'),
    'decoder_h4': tf.Variable(tf.truncated_normal([n_hidden_1,n_input], stddev=0.1), name='decoder_h4'),
}
classify = tf.Variable(tf.truncated_normal([n_hidden_4, n_class], stddev=0.1), tf.float32, name='cf')

biases = {
    'encoder_b1': tf.Variable(tf.random_normal([n_hidden_1]), name='encoder_b1'),
    'encoder_b2': tf.Variable(tf.random_normal([n_hidden_2]), name='encoder_b2'),
    'encoder_b3': tf.Variable(tf.random_normal([n_hidden_3]), name='encoder_b3'),
    'encoder_b4': tf.Variable(tf.random_normal([n_hidden_4]), name='encoder_b4'),
    'decoder_b1': tf.Variable(tf.random_normal([n_hidden_3]), name='decoder_b1'),
    'decoder_b2': tf.Variable(tf.random_normal([n_hidden_2]), name='decoder_b2'),
    'decoder_b3': tf.Variable(tf.random_normal([n_hidden_1]), name='decoder_b3'),
    'decoder_b4': tf.Variable(tf.random_normal([n_input]), name='decoder_b4'),
}
weightse1_summary = tf.summary.histogram("weights_e1", weights['encoder_h1'])
weightse2_summary = tf.summary.histogram("weights_e2", weights['encoder_h2'])
weightse3_summary = tf.summary.histogram("weights_e3", weights['encoder_h3'])
weightse4_summary = tf.summary.histogram("weights_e4", weights['encoder_h4'])

# def encoder(x):  
layer_1 = tf.nn.relu(tf.add(tf.matmul(X, weights['encoder_h1']), biases['encoder_b1']))  
layer_2 = tf.nn.relu(tf.add(tf.matmul(layer_1, weights['encoder_h2']), biases['encoder_b2']))  
layer_3 = tf.nn.relu(tf.add(tf.matmul(layer_2, weights['encoder_h3']), biases['encoder_b3']))
layer_4 = tf.nn.relu(tf.add(tf.matmul(layer_3, weights['encoder_h4']), biases['encoder_b4']))
#     return layer_4
encoder_op = layer_4

def decoder(x):  
    layer_1 = tf.nn.tanh(tf.add(tf.matmul(x, weights['decoder_h4']),  
                                   biases['decoder_b4']))  
#     layer_2 = tf.nn.tanh(tf.add(tf.matmul(layer_1, weights['decoder_h2']),  
#                                    biases['decoder_b2']))  
#     layer_3 = tf.nn.tanh(tf.add(tf.matmul(layer_2, weights['decoder_h3']),
#                                    biases['decoder_b3']))
#     layer_4 = tf.nn.tanh(tf.add(tf.matmul(layer_3, weights['decoder_h4']),
#                                    biases['decoder_b4']))
    return layer_1

def binaryclassify(x):
    out = tf.matmul(x, classify)
    return out

# encoder_op = encoder(X)
py = binaryclassify(encoder_op)
# cross_entropy = tf.reduce_mean(-tf.reduce_sum(y * tf.log(py), reduction_indices=[1]))
# cross_entropy = -tf.reduce_mean(y*tf.log(tf.clip_by_value(py,1e-10,1.0)))
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=py))
# decoder_op = decoder(encoder_op)

# y_pred = predict
# y_true = y

# cost = tf.reduce_mean(tf.pow(y_true-y_pred,2))
# cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_pred, labels=y_true))
cost_summary = tf.summary.scalar('cross_entropy', cross_entropy)  
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, name='Adam').minimize(cross_entropy)


correct_prediction = tf.equal(tf.argmax(py,1), tf.argmax(y, 1))
trainacc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
trainacc_summary = tf.summary.scalar('trainacc', trainacc) 

testacc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
testacc_summary = tf.summary.scalar('testacc', testacc) 

In [4]:
init = tf.global_variables_initializer() 
saver = tf.train.Saver()
sess = tf.Session()
new = 0
total_step = 0
if new == 1:
    sess.run(init)
else:
    init = tf.global_variables_initializer() 
    sess.run(init)
    saver.restore(sess, 'model.ckpt')
    total_step = np.load("./total_step.npy")

INFO:tensorflow:Restoring parameters from model.ckpt


In [5]:
# saver = tf.train.Saver()
# sess = tf.Session()
# # new_saver = tf.train.import_meta_graph('model.ckpt.meta')
# init = tf.global_variables_initializer() 
# sess.run(init)
# saver.restore(sess, 'model.ckpt')

In [6]:
# all_vars = tf.trainable_variables()
# for v in all_vars:
#     print v.name
# sess.run("cf:0")
# #     sess.run(classify)

In [7]:
## change test set to one-hot
test_index = np.random.permutation(test_data.shape[0])
tl_list = test_label[test_index]
tl = np.zeros((test_label.shape[0], 2), dtype=np.int)
line = 0
td = test_data[test_index]
for i in tl_list:
    tl[line][i] = 1
    line += 1

In [8]:
index = np.random.permutation(train_data.shape[0])
def get_batch_np(itert, batchsize):
    batch_index = index[itert*batchsize:(itert+1)*batchsize]
    batch_data = train_data[batch_index]
    label_list = train_label[batch_index]
    batch_label = np.zeros((batchsize, 2), dtype=np.int)
    line = 0
    for i in label_list:
        batch_label[line][i] = 1
        line += 1
    return batch_data, batch_label

In [11]:
BATCH_SIZE = 500
training_epochs = 100
test_display_step = 10
# 首先计算总批数，保证每次循环训练集中的每个样本都参与训练，不同于批量训练  
total_batch = int(train_data.shape[0]/BATCH_SIZE) #总批数  
 
merged_summary_op = tf.summary.merge([cost_summary, weightse1_summary, weightse2_summary, weightse3_summary, weightse4_summary,
                                     trainacc_summary])
summary_writer = tf.summary.FileWriter('./tmp/logs', sess.graph)
summary_writer.add_graph(sess.graph)


for epoch in range(training_epochs):  
    index = np.random.permutation(train_data.shape[0])
    for i in range(total_batch):  
        total_step += 1
        batch_xs, batch_ys= get_batch_np(i, BATCH_SIZE)  # max(x) = 1, min(x) = 0  
        # Run optimization op (backprop) and cost op (to get loss value)
        
        _, c = sess.run([optimizer, cross_entropy], feed_dict={X: batch_xs, y: batch_ys, learning_rate:0.000001})
        
        summary_str, _, c, ta = sess.run([merged_summary_op, optimizer, cross_entropy, trainacc], 
                                               feed_dict={X: batch_xs, y: batch_ys, learning_rate:0.000001})
        summary_writer.add_summary(summary_str, total_step)
        
    if epoch % display_step == 0:  
        record = open("train_loss.txt", "a+")
        record.write("epoch:"+str(epoch)+" loss:"+"{:.9f}\n".format(c))
        record.close()
        print "Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(c) , "trainacc = ", "{:.3f}".format(ta)
    if epoch % test_display_step == 0:
        summary_str, tsta = sess.run([tf.summary.merge([testacc_summary]), testacc], feed_dict={X: td, y: tl})
        summary_writer.add_summary(summary_str, total_step)
        print "TestAcc = ", "{:.3f}".format(tsta)
#         correct_prediction = tf.equal(tf.argmax(py,1), tf.cast(y, tf.int64))
        
#         accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#         batch_xs, batch_ys= get_batch_np(0, 5000)
#         cross_entropy = -tf.reduce_mean(y*tf.log(py))
        # cross_entropy2 = -tf.reduce_mean(tf.cast(y, tf.int64)*tf.log(py))
#         print(sess.run([accuracy, cross_entropy], feed_dict={X: test_data, y: test_label.reshape(1180,1)}))

# sess.run(tf.global_variables_initializer())
# saver = tf.train.Saver()
saver_path = saver.save(sess, "model.ckpt")
print "Model saved in file: ", saver_path
np.save("total_step", total_step)
print("Optimization Finished!")  

Epoch: 0001 cost= 0.024553018 trainacc =  0.990
TestAcc =  0.931
Epoch: 0002 cost= 0.020887913 trainacc =  0.988
Epoch: 0003 cost= 0.031545307 trainacc =  0.982
Epoch: 0004 cost= 0.021554764 trainacc =  0.990
Epoch: 0005 cost= 0.023979973 trainacc =  0.988
Epoch: 0006 cost= 0.029179765 trainacc =  0.984
Epoch: 0007 cost= 0.018840007 trainacc =  0.988
Epoch: 0008 cost= 0.022514639 trainacc =  0.988
Epoch: 0009 cost= 0.038621850 trainacc =  0.972
Epoch: 0010 cost= 0.024272565 trainacc =  0.988
Epoch: 0011 cost= 0.022826971 trainacc =  0.990
TestAcc =  0.929
Epoch: 0012 cost= 0.026118100 trainacc =  0.986
Epoch: 0013 cost= 0.016679393 trainacc =  0.988
Epoch: 0014 cost= 0.030096974 trainacc =  0.992
Epoch: 0015 cost= 0.028767927 trainacc =  0.978
Epoch: 0016 cost= 0.032818168 trainacc =  0.980
Epoch: 0017 cost= 0.020022372 trainacc =  0.990
Epoch: 0018 cost= 0.033997178 trainacc =  0.980
Epoch: 0019 cost= 0.021324193 trainacc =  0.984
Epoch: 0020 cost= 0.022154983 trainacc =  0.984
Epoch:

* Training set acc

In [25]:
correct_prediction = tf.equal(tf.argmax(py,1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
batch_xs, batch_ys= get_batch_np(0, 3000)
# cross_entropy2 = -tf.reduce_mean(tf.cast(y, tf.int64)*tf.log(py))
print(sess.run([accuracy], feed_dict={X: batch_xs, y: batch_ys}))

[0.85500002]


In [10]:
[0.96866667]

[0.96866667]

* Test set acc

In [26]:
correct_prediction = tf.equal(tf.argmax(py,1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
# batch_xs, batch_ys= get_batch_np(0, 4000)
# cross_entropy2 = -tf.reduce_mean(tf.cast(y, tf.int64)*tf.log(py))
print(sess.run([accuracy], feed_dict={X: td, y: tl}))

[0.84830511]


In [ ]:
[0.92033899]

In [28]:
#download teh MNIST data in folder "MNIST_data" that in the same path as this *.py
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

import tensorflow as tf

#图片的占位
x = tf.placeholder(tf.float32, [None, 784])

#系数
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

#softmax层
y = tf.nn.softmax(tf.matmul(x, W) + b)

#用于训练的真实值占位
y_ = tf.placeholder(tf.float32, [None, 10])

#交叉熵：-tf.reduce_sum(y_ * tf.log(y)是一个样本的，外面的tf.reduce_mean是batch的
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

#规定训练的方法：注意：使用GradientDescentOptimizer适合上述的误差项
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

#初始化
init = tf.initialize_all_variables()

sess = tf.Session()
sess.run(init)

#训练
for i in range(5):
  batch_xs, batch_ys = mnist.train.next_batch(2)
  #print batch_xs.shape
  a,_ = sess.run([y, train_step], feed_dict={x: batch_xs, y_: batch_ys})
  print a

#验证，argmax(y,1)是获得y的第一个维度（即每一行）的最大值的位置
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run([accuracy,tf.shape(y)], feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Use `tf.global_variables_initializer` instead.
[[ 0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1]
 [ 0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1]]
[[  1.74958725e-02   1.74958725e-02   1.74958725e-02   1.74958725e-02
    1.74958725e-02   1.74958725e-02   1.74958725e-02   5.23108721e-01
    1.74958725e-02   3.36924314e-01]
 [  1.90278515e-04   1.90278515e-04   1.90278515e-04   1.90278515e-04
    1.90278515e-04   1.90278515e-04   1.90278515e-04   1.68166868e-02
    1.90278515e-04   9.81661081e-01]]
[[  1.77685414e-02   5.68983814e-05   5.68983814e-05   5.68983814e-05
    5.68983814e-05   5.68983814e-05   5.68983814e-05   8.84178877e-01
    9.21878219e-02   5.52334916e-03]
 [  6.64212108e-02   6.22349384e-04   6.22349384e-04   6.22349384e-04
    6.22349384e-04   6.

In [16]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/",one_hot=False)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [18]:
mnist.test.labels.shape

(10000,)